In [10]:
import serial
import time
import re
import pandas as pd
import numpy as np
import pickle

from pyargus.directionEstimation import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#from plotly import __version__
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import cufflinks as cf
#import plotly
from IPython.display import IFrame
import serial.tools.list_ports

#init_notebook_mode(connected=True)

#cf.go_offline()

import pickle
import numpy as np
from pyargus.directionEstimation import *

import math
from collections import deque
from scipy.spatial import distance

import time
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [ ]:
BAUD_RATE = 115200
N_SAMPLES_OF_REF_PERIOD = 8
wavelength = 0.121

d = 0.05  # inter element spacing
M = 2  # number of antenna elements in the antenna system

frequency = 2480.0

# map parameters
width_map = 14.7
height_map = 12.4
kx_locator = 0.532471
ky_locator = 0.558504
x_locator = kx_locator * width_map
y_locator = ky_locator * height_map
z_locator = 0.2
z_beacon = 0.814

SIGMA_FILTER_WINDOW = 5

beacon_position = 4

In [ ]:
def get_port():
    ports = serial.tools.list_ports.comports()

    for port, desc, hwid in sorted(ports):

        if "JLink" in desc:
            return port
        
    return null

def get_serial_data(PORT, BAUD_RATE):

    uart = serial.Serial(port=PORT, baudrate=BAUD_RATE)
    
    uart.timeout = 1 

    if uart.is_open:
        iq_sample = ""
        while True:
            received_data = uart.inWaiting()
            if received_data:
                data = uart.read(received_data)
                
                try:
                    data = data.decode('windows-1252')
                except:
                    continue

                if "Data arrived..." not in data:
                    continue
                else:
                    iq_sample = iq_sample + data
                 
                    if "DF_END" in data:
                        uart.close()

                        return iq_sample
            time.sleep(0.15)
    else:
        print('serial not open')
        
        
def get_raw_iq(serial_data):
    iq = re.split('\r|\n', serial_data)
    
    df = pd.DataFrame(iq, columns=['data'])
    filter = df["data"] != ""
    df = df[filter].reset_index()
    
    start_idx = df[df.data =='Data arrived...'].first_valid_index() + 11
    
    freq = df.data[start_idx - 5]
    freq = float(freq.split("FR:")[1])
    wave_length = (3e8 / (freq*1e6))*1000 #in mm scale
    
    last_indices = df.index[df.data == 'DF_END'].tolist()
    
    last_idx = 0
    for idx in last_indices:
        if idx>start_idx:
            last_idx = idx
            break
    
    if last_idx == 0:
        return
    
    df = df[start_idx:last_idx].reset_index()["data"]
    df = df.str.split(',', expand=True)
    
    df.columns =['IQ', 'Time', 'Antenna', 'Q', 'I']
    df["IQ"] = df["IQ"].str.replace("IQ:", "")
    
    ret_df = df.astype(str).astype(int)
    
    ret_df["Frequency"] = freq
    ret_df["Wavelength"] = wave_length
    
    return ret_df

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def get_antenna_iq(iq_samples, antenna):
    df = iq_samples[iq_samples['Antenna']==antenna]
    iq = list(zip(df.I, df.Q))
    
    return iq

def get_angle(X):
    # Estimating the spatial correlation matrix
    R = corr_matrix_estimate(X.T, imp="fast")

    array_alignment = np.arange(0, M, 1) * d
    incident_angles = np.arange(-90, 91, 1)
    scanning_vectors = np.zeros((M, np.size(incident_angles)), dtype=complex)
    for i in range(np.size(incident_angles)):
        scanning_vectors[:, i] = np.exp(
            array_alignment * 1j * 2 * np.pi * np.sin(np.radians(incident_angles[i])) / wavelength)  # scanning vector

    ula_scanning_vectors = scanning_vectors

    # Estimate DOA
    MUSIC = DOA_MUSIC(R, ula_scanning_vectors, signal_dimension=1)
    norm_data = np.divide(np.abs(MUSIC), np.max(np.abs(MUSIC)))
    return float(incident_angles[np.where(norm_data == 1)[0]][0])


class SigmaFilter:
    def __init__(self, maxlen = SIGMA_FILTER_WINDOW):
        self.deque = deque(maxlen = maxlen)

    def isValid(self, v):
        isValid = False
        if len(self.deque) == SIGMA_FILTER_WINDOW:
            mean_v, std_v = np.array(self.deque).mean(axis=0), np.array(self.deque).std(axis=0)
            if distance.euclidean(v, mean_v) < distance.euclidean(mean_v + 3 * std_v, mean_v):
                isValid = True
            else:
                print('outlier:', v)

        self.deque.append(v)
        return isValid

def get_coordinate(azimuth, elevation, height, receiver_coords):
    nx = np.cos(np.deg2rad(90.0 - azimuth))
    nz = np.cos(np.deg2rad(90.0 - abs(elevation)))
    if math.isclose(nx, 0.0, abs_tol=1e-16) or math.isclose(nz, 0.0, abs_tol=1e-16):
        return [float("nan"),  float("nan")]
    else:
        ny = np.sqrt(1 - nx ** 2 - nz ** 2)
        t = (height - receiver_coords[2]) / nz
        x = receiver_coords[0] + t * nx
        y = receiver_coords[1] - t * ny
    return [x, y]

def get_camera_coordinate(azimuth, elevation, focal_length, pixel_size):

    azimuth = np.deg2rad(azimuth)
    elevation = np.deg2rad(elevation)

    k1 = pixel_size[0]*focal_length
    k2 = pixel_size[1]*focal_length
    
    try:
        n = int(k1*(np.cos(azimuth)/np.sin(azimuth)))
        m = int(k2*(np.cos(elevation)/(np.sin(azimuth)*np.sin(elevation))))
    
    except:
        return [-1, -1]
    
    return [n,m]
    

In [ ]:
# 0 -> Antenna A1.1
# 1 -> Antenna A1.2
# 2 -> Antenna A1.3
# 3 -> Antenna A2.1
# 4 -> Antenna A2.2
# 5 -> Antenna A2.3 , reference anteanna


velSigmaFilter = SigmaFilter(SIGMA_FILTER_WINDOW)

# x_pixel = []
# y_pixel = []

azimuths = []
elevations = []

while(1):

    aoa_dfs = []

    serial_data = get_serial_data(get_port(), BAUD_RATE)
    
    try:
        df = get_raw_iq(serial_data)
    
        df['Beacon Position'] = beacon_position

    except:
        continue
        
    aoa_dfs.append(df)

    for iq_samples in aoa_dfs:

        if iq_samples['Frequency'][0] != frequency:
            continue

        ref_phases = []
        azimuth_phases, elevation_phases = [], []
        x_00, y_00 = [], []

        for iq_idx in range(N_SAMPLES_OF_REF_PERIOD - 1):
            iq_next = complex(iq_samples["I"][iq_idx + 1], iq_samples["Q"][iq_idx + 1])
            iq_cur = complex(iq_samples["I"][iq_idx], iq_samples["Q"][iq_idx])
            phase_next = np.rad2deg(np.arctan2(iq_next.imag, iq_next.real))
            phase_cur = np.rad2deg(np.arctan2(iq_cur.imag, iq_cur.real))
            ref_phases.append((to_plus_minus_pi(phase_next - phase_cur)))
        phase_ref = np.mean(ref_phases)

        iq_1 = get_antenna_iq(iq_samples, antenna = 1)
        iq_2 = get_antenna_iq(iq_samples, antenna = 2)
        iq_3 = get_antenna_iq(iq_samples, antenna = 3)
        iq_4 = get_antenna_iq(iq_samples, antenna = 4)

        azimuth_iq = list(zip(iq_1, iq_2))
        elevation_iq = list(zip(iq_3, iq_4))

        for iq in azimuth_iq:
            iq_cur = complex(iq[0][0],iq[0][1])
            iq_next = complex(iq[1][0], iq[1][1])

            phase_next = np.rad2deg(np.arctan2(iq_next.imag, iq_next.real))
            phase_cur = np.rad2deg(np.arctan2(iq_cur.imag, iq_cur.real))
            diff_phase = to_plus_minus_pi((phase_next - phase_cur) - 2 * phase_ref)

            azimuth_phases.append(diff_phase)

            x_00.append(1)

        for iq in elevation_iq:
            iq_cur = complex(iq[0][0],iq[0][1])
            iq_next = complex(iq[1][0], iq[1][1])

            phase_next = np.rad2deg(np.arctan2(iq_next.imag, iq_next.real))
            phase_cur = np.rad2deg(np.arctan2(iq_cur.imag, iq_cur.real))
            diff_phase = to_plus_minus_pi((phase_next - phase_cur) - 2 * phase_ref)

            elevation_phases.append(diff_phase)

            y_00.append(1)

        azimuth_x_12, elevation_x_12 = [], []

        X = np.zeros((M, np.size(x_00)), dtype=complex)
        X[0, :] = x_00
        for i in azimuth_phases:
            azimuth_x_12.append(np.exp(1j * np.deg2rad(i)))

        X[1, :] = azimuth_x_12
        azimuth_angle = get_angle(X)

        Y = np.zeros((M, np.size(y_00)), dtype=complex)
        Y[0, :] = y_00

        for i in elevation_phases:
            elevation_x_12.append(np.exp(1j * np.deg2rad(i)))
        Y[1, :] = elevation_x_12
        elevation_angle = get_angle(Y)
        
        azimuths.append(azimuth_angle)
        elevations.append(elevation_angle)
        
        clear_output(wait=True)
       
        plt.xlim([0, 200])
        plt.ylim([-200, 200])
        
    
        plt.scatter([*range(1,len(azimuths)+1)], azimuths) 
        
        #plt.scatter([*range(1,len(elevations)+1)], elevations) 
        
        plt.show()
        
#         focal_length = 26.0e-3
#         pixel_size = [2532, 1170]
        
#         xy = get_camera_coordinate(azimuth_angle, elevation_angle, focal_length, pixel_size)
        
#         x_pixel.append(xy[0])
#         y_pixel.append(xy[1])
        
        
#         clear_output(wait=True)
       
#         plt.scatter(x_pixel, y_pixel)
#         plt.xlim([0, pixel_size[0]])
#         plt.ylim([0, pixel_size[1]])
    
#         plt.show()